## Reading SEG2 files from shotsgahters and sort traces as CMP gather
Strategy: read all files in a loop, pack all x-t-matrices into cube with dimensions of time reciever positions and source positions.
Then we count up the diagonals of the matrix of source postions and index and the anty-diagonal thrue the matrix of recievers.

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from obspy import read  #read function
import os

In [15]:
Datadir='Leine6-7-23/'  #type in here the data filefolder which have to be in the recent folder of this notebook
fileNames=os.listdir(Datadir) #that'll gives a list of all files ind the folder with that name
fileNames = [i for i in fileNames if i.endswith('.dat')] #select only data files
fileNames.sort()   #sort them bei title number what accords to the shotnumber which you might recunstruct where you did the shot

In [16]:
nf=len(fileNames) # Number of files is number of shots
seg2_data = read(Datadir+fileNames[0], format='SEG2')  # Read the first file of the first shot 
nt=len(seg2_data[0].data) #to get number of time samples, assumung that for all data in data set. Otherway pray that the field operator did not change parameters over the dataset
nx = len(seg2_data) #number of all traces in one file
shotga = np.zeros((nt, nx)) #allocating matrix as shotgather
reclox=np.zeros((nx))  #and for the reciever locations
srcloc=np.zeros((nf))  #and for source locations

/home/goeran/.local/lib/python3.10/site-packages/obspy/io/seg2/seg2.py:365: UserWarning: Many companies use custom defined SEG2 header variables. This might cause basic header information reflected in the single traces' stats to be wrong (e.g. recording delays, first sample number, station code names, ..). Please check the complete list of additional unmapped header fields that gets stored in Trace.stats.seg2 and/or the manual of the source of the SEG2 files for fields that might influence e.g. trace start times.
  warnings.warn(WARNING_HEADER)


In [17]:
Cube=np.zeros((nt,nx,nf)) #allocation a 3D array wit dimensions_ timesamples, recieverpositions and shotpositions
for fi in range(0,nf):# Loop through each SEG2 file representing a shot
    seg2_data = read(Datadir+fileNames[fi], format='SEG2')  # Load the SEG2 file   
    #tracy =np.zeros((nt,1))
    shotga = np.zeros((nt, nx))
    # Iterate through the traces for the current shot and store their data
    for j in range(nx):

        tracy=seg2_data[j].data #a trace        
        Hff=np.linspace(0,np.max(tracy),nt)#**2 #some amplitude spherical correction
        tracy=tracy*Hff
        tracy=tracy/(np.max(tracy))
        shotga[:,j] = tracy   
        reclox[j] = float(seg2_data[j].stats.seg2['RECEIVER_LOCATION'])
    srcloc[fi]=float(seg2_data[0].stats.seg2['SOURCE_LOCATION'])#should not change
    Cube[:,:,fi]=shotga

In [18]:
dt=float(seg2_data[j].stats.delta) #if you like seconds...
np.shape(Cube) #voila the cube
np.savez('Cube_tata', Cube)

In [7]:
#Ok we shift all positions to positiv values, especially when we count forward and back thats more clear.
#source locations, starting from zero and the receiver positions accordingly
srcloz=srcloc-np.min(srcloc) #[  0.,   4.,   8.,  12., ....... 180., 184., 188., 192., 196., 200.]
recloz=reclox-np.min(srcloc) #[ 52.,  56.,  60.,  64., .....  228., 232., 236., 240.]

In [11]:
rm, sm = np.meshgrid(recloz,srcloz) #blowing up matrizes with positions 
ri, si = np.meshgrid(np.arange(0, nx, 1),np.arange(0, nf, 1))# nf X nx matrizen
[np.shape(ri),np.shape(rm)]

[(51, 48), (51, 48)]

In [ ]:
for sii in range(-nx,nf-(nf-nx)-1):           #counting all diagonals of the kordinaten and index matrizes
    disrc=np.diagonal(sm,sii)                #diagonal elements form source-matrix (sm)
    direc=np.diagonal(np.fliplr(rm),sii)     #diagonals from reciever-matrix (rm) but in reversed order .... same indexes produce Common offesets

    disrci=np.diagonal(si,sii)            # the indizes of sources 
    direci=np.diagonal(np.fliplr(ri),sii) #
    
    Zoffesets=(direc-disrc)/2            # offests from midpoint
    mip=disrc+Zoffesets                   # midpoint as a vector of all offsets to check, are alle the same... one should be enough
    #### uncomment and use this printfunctions to get all combinations of possible CMPS  
    #print(sii,Zoffesets, np.mean(mip))
    #print(sii,disrci,direci)
    fig = plt.figure(figsize=(10, 5))
    ax = fig.add_subplot(1, 2, 1)
    ax.set_title(f'Midpoint at: {np.mean(mip)}')
    ax.set_xlabel('reciever postion (m)')
    ax.set_ylabel('time (s)')
    #cmap = ax.imshow(Cube[:,direci,disrci], aspect='auto',interpolation='none')
    cmap = ax.pcolormesh(direc, np.linspace(0,dt*nt,nt), Cube[:,direci,disrci])#, vmin= -0.1, vmax= 0.1)
    plt.gca().invert_yaxis()
    #plt.gca().invert_xaxis()
    fig.colorbar(cmap)
    

    